# Aritificial intelligence assignment 2024

In dit document wordt een model getraind op de mnist nummers dataset. Het model bevat 3 lagen (Flatten, Dense, Dense/output) 
waar nog mee geexperimenteerd is. Daarnaast wordt de standaard accuracy getoond voor het meten van de nauwkeurigheid van het model.
De loss function sparse_categorical_crossentropy wordt gebruikt en de nauwkeurigheid van het model is het hoogst met epochs = 5 en 
validation_split=0.1. Dit betekend dat de dataset 5x door het model heen gehaald is met forward- en backward propogation.
De validation_split van 0.1 betekend dat 1/100e van de trainingsset als validatie data/held out data dient.

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

mnist = tf.keras.datasets.mnist

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.1


In [21]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [23]:
# Normalize the pixel values to be between 0 and 1
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [41]:
# Reshape the data if using convolutional neural networks (CNNs)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [43]:
# Toon de data
print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

Training data shape: (60000, 28, 28, 1)
Test data shape: (10000, 28, 28, 1)


In [45]:
model = Sequential()

#Maak de 28x28 array plat tot een 1 dimentionale array of een lijst
model.add(Flatten(input_shape=(28, 28)))

#Check wat efficienter is voor de nauwkeurigheid van het model, Conv2D or double Dense layers
#1: model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
#2: model.add(Dense(128, activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

#Compile the current model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train het model met validatie
history = model.fit(x_train, y_train, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8579 - loss: 0.4968 - val_accuracy: 0.9672 - val_loss: 0.1164
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9632 - loss: 0.1209 - val_accuracy: 0.9688 - val_loss: 0.0945
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9765 - loss: 0.0762 - val_accuracy: 0.9757 - val_loss: 0.0811
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9829 - loss: 0.0549 - val_accuracy: 0.9782 - val_loss: 0.0761
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9867 - loss: 0.0407 - val_accuracy: 0.9760 - val_loss: 0.0837


In [17]:
# Evalueer het model op de testset
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f"Test nauwkeurigheid: {test_accuracy:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9684 - loss: 0.1108
Test nauwkeurigheid: 0.9725


In [21]:
# Modelgrootte berekenen
model_size = model.count_params()
model_memory = model_size * 4 / (1024 ** 2)  # 4 bytes per 32-bit float
print(f"Modelgrootte: {model_memory:.2f} MB")

Modelgrootte: 0.45 MB


# In dit deel wordt een nauwkeurigheid van het model getoond door middel van de confusion matrix

In [24]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [26]:
# Maak voorspellingen op de testset
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [28]:
# Bereken de verwarringsmatrix
cm = confusion_matrix(y_test, y_pred_classes)
print("Verwarringsmatrix:")
print(cm)

Verwarringsmatrix:
[[ 958    0    2    2    2    5    5    1    3    2]
 [   0 1121    1    2    0    1    2    1    7    0]
 [   1    4  970   24    4    0    4   11   14    0]
 [   0    0    0  997    0    5    0    2    6    0]
 [   0    1    3    0  963    0    2    3    2    8]
 [   2    0    0   13    0  864    4    0    7    2]
 [   4    3    0    1    5    3  937    0    5    0]
 [   2    2    4    7    2    0    0 1000    2    9]
 [   0    1    2    9    2    3    1    3  948    5]
 [   3    4    0    9   16    4    0    3    3  967]]


De verwarringsmatrix is te lezen als volgt:
De x as moeten de getallen van 0 tm 9 bevatten die het model voorspelt.
De y as moeten de getallen zijn die gelabeld zijn als 0 tm 9.

In deze matrix kunnen we zien dat klasse 8 op de y-as het meest fout gedaan wordt door het model. Daarnaast kan hieruit geconcludeerd worden dat klasse 7 op de x as het meest fout geraden is door het model en het model hier dus moeite mee heeft.